# Tablesaw 

[Tablesaw](https://tablesaw.tech/) is easy to add to the BeakerX Groovy kernel.
Tablesaw provides the ability to easily transform, summarize, and filter data, as well as computing descriptive statistics and fundamental machine learning algorithms.

This notebook has some basic demos of how to use Tablesaw, including visualizing the results.  This notebook uses the Beaker interactive visualizaiton libraries, but Tablesaw's APIs also work.  The notebook covers basic table manipulation, k-means clustering, and linear regression.

In [28]:
%%classpath add mvn
tech.tablesaw tablesaw-plot 0.11.4
tech.tablesaw tablesaw-smile 0.11.4
tech.tablesaw tablesaw-beakerx 0.11.4

In [29]:
%import tech.tablesaw.aggregate.*
%import tech.tablesaw.api.*
%import tech.tablesaw.api.ml.clustering.*
%import tech.tablesaw.api.ml.regression.*
%import tech.tablesaw.columns.*

// display Tablesaw tables with BeakerX table display widget
tech.tablesaw.beakerx.TablesawDisplayer.register()

null

In [30]:
tornadoes = Table.read().csv("../resources/data/tornadoes_2014.csv")

In [31]:
//print dataset structure
tornadoes.structure()

In [32]:
//get header names
tornadoes.columnNames()

[Date, Time, State, State No, Scale, Injuries, Fatalities, Start Lat, Start Lon, Length, Width]

In [33]:
//displays the row and column counts
tornadoes.shape()

908 rows X 11 cols

In [34]:
//displays the first n rows
tornadoes.first(10)

In [35]:
import static tech.tablesaw.api.QueryHelper.column
tornadoes.structure().selectWhere(column("Column Type").isEqualTo("FLOAT"))

In [36]:
//summarize the data in each column
tornadoes.summary()


Table summary for: tornadoes_2014.csv
       Column: Date        
 Measure   |    Value     |
---------------------------
    Count  |         908  |
  Missing  |           0  |
 Earliest  |  2014-01-11  |
   Latest  |  2014-12-29  |
     Column: Time     
 Measure   |  Value  |
----------------------
    Count  |    908  |
  Missing  |      0  |
 Earliest  |  00:01  |
   Latest  |  23:59  |
    Column: State     
 Category  |  Count  |
----------------------
       GA  |     32  |
       NM  |     15  |
       MT  |      8  |
       CO  |     49  |
       WV  |      9  |
       IN  |     28  |
       MD  |      2  |
       CA  |      9  |
       AL  |     55  |
       TN  |     18  |
      ...  |    ...  |
       MO  |     47  |
       ME  |      4  |
       LA  |     15  |
       MI  |     13  |
       SC  |      7  |
       KY  |     28  |
       MA  |      3  |
       CT  |      1  |
       NH  |      2  |
   Column: State No   
 Measure   |  Value  |
----------------------
      

In [37]:
//Mapping operations
def month = tornadoes.dateColumn("Date").month()
tornadoes.addColumn(month);
tornadoes.columnNames()

[Date, Time, State, State No, Scale, Injuries, Fatalities, Start Lat, Start Lon, Length, Width, Date month]

In [38]:
//Sorting by column
tornadoes.sortOn("-Fatalities")

In [39]:
//Descriptive statistics
tornadoes.column("Fatalities").summary()

In [40]:
//Performing totals and sub-totals
def injuriesByScale = tornadoes.median("Injuries").by("Scale")
injuriesByScale.setName("Median injuries by Tornado Scale")
injuriesByScale

In [41]:
//Cross Tabs
CrossTab.xCount(tornadoes, tornadoes.categoryColumn("State"), tornadoes.shortColumn("Scale"))

## K-means clustering

K-means is the most common form of “centroid” clustering. Unlike classification, clustering is an unsupervised learning method. The categories are not predetermined. Instead, the goal is to search for natural groupings in the dataset, such that the members of each group are similar to each other and different from the members of the other groups. The K represents the number of groups to find.

We’ll use a well known Scotch Whiskey dataset, which is used to cluster whiskeys according to their taste based on data collected from tasting notes. As always, we start by loading data and printing its structure.

More description is available at https://jtablesaw.wordpress.com/2016/08/08/k-means-clustering-in-java/

In [42]:
t = Table.read().csv("../resources/data/whiskey.csv")
t.structure()

In [43]:
model = new Kmeans(
    5,
    t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
    t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
);

//print claster formation
model.clustered(t.column("Distillery"));

In [44]:
//print centroids for each claster
model.labeledCentroids();

In [45]:
//gets the distortion for our model
model.distortion()

380.46206896551723

In [46]:
def n = t.rowCount();
def kValues = new double[n - 2];
def distortions = new double[n - 2];

for (int k = 2; k < n; k++) {
  kValues[k - 2] = k;
  def kmeans = new Kmeans(k,
      t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
      t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
  );
  distortions[k - 2] = kmeans.distortion();
}
def linearYPlot = new Plot(title: "K-means clustering demo", xLabel:"K", yLabel: "distortion")
linearYPlot << new Line(x: kValues, y: distortions)

## Play (Money)ball with Linear Regression

In baseball, you make the playoffs by winning more games than your rivals. The number of games the rivals win is out of your control so the A’s looked instead at how many wins it took historically to make the playoffs. They decided that 95 wins would give them a strong chance.  Here’s how we might check that assumption in Tablesaw.

More description is available at https://jtablesaw.wordpress.com/2016/07/31/play-moneyball-data-science-in-tablesaw/

In [47]:
import static tech.tablesaw.api.QueryHelper.column

baseball = Table.read().csv("../resources/data/baseball.csv");

// filter to the data available at the start of the 2002 season
moneyball = baseball.selectWhere(column("year").isLessThan(2002));
wins = moneyball.nCol("W");
year = moneyball.nCol("Year");
playoffs = moneyball.column("Playoffs");
runDifference = moneyball.shortColumn("RS").subtract(moneyball.shortColumn("RA"));
moneyball.addColumn(runDifference);
runDifference.setName("RD");

def Plot = new Plot(title: "RD x Wins", xLabel:"RD", yLabel: "W")
Plot << new Points(x: moneyball.numericColumn("RD").toDoubleArray(), y: moneyball.numericColumn("W").toDoubleArray())

In [48]:
winsModel = LeastSquares.train(wins, runDifference);

Linear Model:

Residuals:
	       Min	        1Q	    Median	        3Q	       Max
	  -14.2662	   -2.6511	    0.1282	    2.9365	   11.6570

Coefficients:
            Estimate        Std. Error        t value        Pr(>|t|)
(Intercept)    80.8814            0.1312       616.6747          0.0000 ***
RD              0.1058            0.0013        81.5536          0.0000 ***
---------------------------------------------------------------------
Significance codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.9391 on 900 degrees of freedom
Multiple R-squared: 0.8808,    Adjusted R-squared: 0.8807
F-statistic: 6650.9926 on 1 and 900 DF,  p-value: 0.000


In [49]:
def runDiff = new double[1];
runDiff[0] = 135;
def expectedWins = winsModel.predict(runDiff);
runsScored2 = LeastSquares.train(moneyball.nCol("RS"), moneyball.nCol("OBP"), moneyball.nCol("SLG"));

Linear Model:

Residuals:
	       Min	        1Q	    Median	        3Q	       Max
	  -70.8379	  -17.1810	   -1.0917	   16.7812	   90.0358

Coefficients:
            Estimate        Std. Error        t value        Pr(>|t|)
(Intercept)  -804.6271           18.9208       -42.5261          0.0000 ***
OBP          2737.7682           90.6846        30.1900          0.0000 ***
SLG          1584.9085           42.1556        37.5966          0.0000 ***
---------------------------------------------------------------------
Significance codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 24.7900 on 899 degrees of freedom
Multiple R-squared: 0.9296,    Adjusted R-squared: 0.9294
F-statistic: 5933.7256 on 2 and 899 DF,  p-value: 0.000


In [50]:
new Histogram(xLabel:"X",
              yLabel:"Proportion",
              data: Arrays.asList(runsScored2.residuals()), 
              binCount: 25);

## Financial and Economic Data

You can fetch data from [Quandl](https://www.quandl.com/) and load it directly into Tablesaw

In [55]:
%%classpath add mvn
com.jimmoores quandl-tablesaw 2.0.0
javax.activation activation 1.1.1

In [56]:
%import com.jimmoores.quandl.*
%import com.jimmoores.quandl.tablesaw.*

In [57]:
TableSawQuandlSession session = TableSawQuandlSession.create();
Table table = session.getDataSet(DataSetRequest.Builder.of("WIKI/AAPL").build());
// Create a new column containing the year
ShortColumn yearColumn = table.dateColumn("Date").year();
yearColumn.setName("Year");
table.addColumn(yearColumn);
// Create max, min and total volume tables aggregated by year
Table summaryMax = table.groupBy("year").max("Adj. Close");
Table summaryMin = table.groupBy("year").min("Adj. Close");
Table summaryVolume = table.groupBy("year")sum("Volume");
// Create a new table from each of these
summary = Table.create("Summary", summaryMax.column(0), summaryMax.column(1), 
                       summaryMin.column(1), summaryVolume.column(1));
// Add back a DateColumn to the summary...will be used for plotting
DateColumn yearDates = new DateColumn("YearDate");
for(year in summary.column('Year')){
    yearDates.append(java.time.LocalDate.of(year,1,1));
}
summary.addColumn(yearDates)

summary

In [58]:
years = summary.column('YearDate').collect()

plot = new TimePlot(title: 'Price Chart for AAPL', xLabel: 'Time', yLabel: 'Max [Adj. Close]')
plot << new YAxis(label: 'Volume')
plot << new Points(x: years, y: summary.column('Max [Adj. Close]').collect())
plot << new Line(x: years, y: summary.column('Max [Adj. Close]').collect(), color: Color.blue)
plot << new Stems(x: years, y: summary.column('Sum [Volume]').collect(), yAxis: 'Volume')